In [1]:
pip install langchain_ollama

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install langchain==1.0.5 langchain-community==0.4.1 langchain-text-splitters==1.0.0

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
pip install jq

  Obtaining dependency information for jq from https://files.pythonhosted.org/packages/a5/1d/2af863d11a5330b69af6cc875bb54ecf942da4909b75284afef7468e70b5/jq-1.10.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/421.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/421.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/421.7 kB ? eta -:--:--
    --------------------------------------- 10.2/421.7 kB ? eta -:--:--
    --------------------------------------- 10.2/421.7 kB ? eta -:--:--
   -- ------------------------------------ 30.7/421.7 kB 186.2 kB/s eta 0:00:03
   -- ------------------------------------ 30.7/421.7 kB 186.2 kB/s eta 0:00:03
   --- ----------------------------------- 41.0/421.7 kB 151.3 kB/s eta 0:00:03
   ----- --------------------------------- 61.4/421.7 kB 192.5 kB/s eta 0:00:02
   ----- --------------------------------- 61.4/421.7 kB 192.5 kB/s eta 0:00:02
   ------ -------------------------------- 7


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_classic.chains import RetrievalQA
from langchain_community.document_loaders import JSONLoader

In [29]:
loader = JSONLoader(
    file_path=r"C:\Users\gangeshvar.s\Desktop\langchain\retrievedoutput.json",
    jq_schema=".query, .retrieved_chunks[]",       # to extract values of both query key and retrieved_chunks key where retrieved_chunks is a list
    text_content=True,                  #true for plain text , false for nested json
)

In [30]:
docs = loader.load()

for i, doc in enumerate(docs, start=1):
    print(f"--- Document {i} ---")
    print(doc.page_content)

--- Document 1 ---
Retrieve and report the latest available vital signs for the patient. Include: allergies, blood pressure, pulse, respiratory rate, SpO₂, and weight. Provide numerical values when applicable.
--- Document 2 ---
, MD Service: — Author Type: Physician
Filed: 10/9/2024 1:17 PM Encounter Date: 10/9/2024 Creation Time: 10/9/2024 8:31 AM
Status: Sign when Signing Visit Editor: Callegan-Poche, Amanda L
--- Document 3 ---
Review of patient's allergies indicates:
No Known Allergies
Physical Examination
Vitals:
10/09/24 0823
BP: 133/63
Pulse: 62
Resp: 16
SpO2: (!) 94%
Weight: 84
--- Document 4 ---
Physical Exam
Health Maintenance / Imaging / Labs
Generated on 10/30/24 11:39 AM Page 5
--- Document 5 ---
Medical / Social / Family History
Past Medical History:
Diagnosis Date
•Diabetes mellitus, type 2
•Hyperlipidemia
•Hypertension
No past surgical history on file
--- Document 6 ---
8 04/08/2024
AST 28 04/08/2024
ALT 36 04/08/2024
Assessment
William F Melvin is a 76 y


In [31]:
query=docs[0].page_content
print(query)

Retrieve and report the latest available vital signs for the patient. Include: allergies, blood pressure, pulse, respiratory rate, SpO₂, and weight. Provide numerical values when applicable.


In [32]:
# 2) Split into chunks for embedding
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs[1:])

In [33]:
chunks

[Document(metadata={'source': 'C:\\Users\\gangeshvar.s\\Desktop\\langchain\\retrievedoutput.json', 'seq_num': 2}, page_content=', MD Service: — Author Type: Physician\nFiled: 10/9/2024 1:17 PM Encounter Date: 10/9/2024 Creation Time: 10/9/2024 8:31 AM\nStatus: Sign when Signing Visit Editor: Callegan-Poche, Amanda L'),
 Document(metadata={'source': 'C:\\Users\\gangeshvar.s\\Desktop\\langchain\\retrievedoutput.json', 'seq_num': 3}, page_content="Review of patient's allergies indicates:\nNo Known Allergies\nPhysical Examination\nVitals:\n10/09/24 0823\nBP: 133/63\nPulse: 62\nResp: 16\nSpO2: (!) 94%\nWeight: 84"),
 Document(metadata={'source': 'C:\\Users\\gangeshvar.s\\Desktop\\langchain\\retrievedoutput.json', 'seq_num': 4}, page_content='Physical Exam\nHealth Maintenance / Imaging / Labs\nGenerated on 10/30/24 11:39 AM Page 5'),
 Document(metadata={'source': 'C:\\Users\\gangeshvar.s\\Desktop\\langchain\\retrievedoutput.json', 'seq_num': 5}, page_content='Medical / Social / Family Histor

In [22]:
# 3) Use Ollama embeddings (default: "nomic-embed-text")
emb = OllamaEmbeddings(model="nomic-embed-text")

In [23]:
# 4) Build FAISS vectorstore
vectorstore = FAISS.from_documents(chunks, emb)

retriever = vectorstore.as_retriever()

In [24]:
# 5) Create TinyLlama LLM
llm = OllamaLLM(model="tinyllama")

In [25]:
# 6) Build QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever,
    return_source_documents=False    
)

In [34]:
answer = qa_chain.invoke(query)

print("\nANSWER:\n", answer)


ANSWER:
 {'query': 'Retrieve and report the latest available vital signs for the patient. Include: allergies, blood pressure, pulse, respiratory rate, SpO₂, and weight. Provide numerical values when applicable.', 'result': 'The latest available vital signs for the patient are as follows:\nAllergies: No allergy detected.\nPhysical Examination: Vital signs recorded as per:\n- BP: 133/63\n- Pulse: 62\n- Respiration rate: 16\n- SpO₂ (breathing): 94%\nWeight: 84 kg\n\nMD Service: MD Author Type is Physician, Filed on: 10/9/2024 1:17 PM, Encounter Date: 10/9/2024, Creation Time: 10/9/2024 8:31 AM, Status: Sign when Signing Visit, Editor: Callega-Poche, Amaandla L. Medical / Social / Family History: Paast medical history: Diagnosis date for hyperlipidemias, hyperthyroidism, and diabetes mellitus type 2, no surgical history on file. No past surgical history recorded on file.'}
